# Python implementation of L-Galaxies

This is a playground to test out the possibility of using `python` as an interface into L-Galaxies.

Abbreviations used:
* desc   – descendent
* gal(s) – galax(y|ies)
* prog   – progenitor
* sub(s) – subhalo(s)

List index indentifiers:
* _gid – relative to the graph
* _sid - relative to the snap

– galaxies/orphans do not need an index identifier as they are numpy arrays defined per snap

Ptyhon type identifiers:
* C_ - class
* D_ - numpy dtype
* F_ - function
* b_ - boolean variable
* c_ - constant (value may be set during parameter initialisation)
* [ijk]_ - variable counter (integer)
* n_ - total counter (integer)

In [ ]:
# Imports of generic python routines

%load_ext autoreload
%autoreload 2

import astropy.constants as c
import astropy.units as u
import gc
import h5py
h5py.enable_ipython_completer()
import numpy as np
import sys

# Switch on traceback for warnings
# import traceback
import warnings

def warn_with_traceback(message, category, filename, lineno, file=None, line=None):

    log = file if hasattr(file,'write') else sys.stderr
    traceback.print_stack(file=log)
    log.write(warnings.formatwarning(message, category, filename, lineno, line))

warnings.showwarning = warn_with_traceback

In [ ]:
# Location of code
C_DIR='code-C'
PYTHON_DIR='code-python'

# Development limiter
n_GRAPH=np.inf
#n_GRAPH=10       # Change output files to 'test' to avoid over-writing!

# Verbosity
VERBOSITY=1 # 1 - Major program steps only; 1/2 - Major/minor Counters; 3/4/5 - Debugging diags.

# List of runtime parameters
FILE_PARAMETERS='input/input.yml'

### Imports of parameter and data classes

In [ ]:
# Imports of py-galaxies python routines
sys.path.insert(1,PYTHON_DIR)

# The parameter class, used to store run-time parameters
from parameters import C_parameters

# The graph class, used to store graphs for processing
from graphs import C_graph

# The halo class, used to store halo properties
from halos import C_halo

# The halo_output class and methods used to output halos
from halos import C_halo_output

# The subhalo class, used to store subhalo properties
from subs import C_sub

# The subhalo_output class, used to output subhalos
from subs import C_sub_output

# The galaxy_output class, used to output galaxies
from gals import C_gal_output

### Initialisation of SAM

In [ ]:
# Read in parameters from yaml input files
parameters=C_parameters(FILE_PARAMETERS,)
parameters.verbosity=VERBOSITY

# Open graph input file: needs to come before F_update_parameters
graph_file=h5py.File(parameters.graph_file,'r')
# Update parameters with attributes from graph_file
parameters.F_update_parameters(graph_file)

# Create counter to locate graphs within the galaxy output file
n_graph=min(parameters.n_graph,n_GRAPH)
n_gal_graph_start=np.full(n_graph,parameters.NO_DATA_INT,dtype=np.int32)
n_gal=0

# Create cooling table
import cooling
cooling_table = cooling.C_cooling(parameters)
# Not sure if this is the besst way to do it, but for now store all globals in parameters
parameters.cooling_table=cooling_table
    
# Create galaxy template
from gals import F_gal_template
gal_template=F_gal_template(parameters)
# Not sure if this is the best way to do it, but for now store all globals in parameters
parameters.gal_template=gal_template

# Create output buffers
halo_output=C_halo_output(parameters)
sub_output=C_sub_output(parameters)
gal_output=C_gal_output(parameters)

# Import driver routines
from driver import F_update_halos     # Propagates info from last snapshot to current one
from driver import F_process_halos    # Does all the astrophysics

### Loop over graphs, snapshots, halos, implementing the SAM

Note that loops over graphs can be done in parallel.

Also, F_update_halos needs to be serial, but all halos can be processed in parallel in F_process_halos.

In [ ]:
# Loop over graphs
for i_graph in range(n_graph):
    if VERBOSITY >= 1: print('Processing graph',i_graph,flush=True)
    graph = C_graph(i_graph,graph_file,parameters)
    
    # Keep track of location in galaxy output file
    n_gal_graph_start[i_graph]=n_gal
    
    # Loop over snapshots
    halos_last_snap = None
    subs_last_snap = None
    gals_last_snap = None
    for i_snap in graph.snap_ID:
        if i_snap == parameters.NO_DATA_INT: 
            assert halos_last_snap == None
            continue
        if VERBOSITY >= 2: print('Processing snapshot',i_snap,flush=True)
            
        # Initialise halo and subhalo properties.
        # This returns a list of halo and subhalo instances
        # This may be slow: an alternative would be to use np arrays.
        halos_this_snap = [C_halo(i_graph,i_snap,i_halo,graph,parameters) for i_halo in 
                         graph.halo_start_gid[i_snap]+range(graph.n_halo_snap[i_snap])]
        subs_this_snap = None
        if graph.n_sub > 0:
            if graph.n_sub_snap[i_snap] > 0:
                subs_this_snap = [C_sub(i_graph,i_snap,i_sub,graph,parameters) 
                                     for i_sub in graph.sub_start_gid[i_snap]+range(graph.n_sub_snap[i_snap])]
        
        # Propagate information from progenitors to this generation
        # Done as a push rather than a pull because sharing determined by progenitor
        # Have to do this even if no progenitors in order to initialise galaxy array
        gals_this_snap=F_update_halos(halos_last_snap,halos_this_snap,subs_last_snap,
                                          subs_this_snap,gals_last_snap,graph,parameters)
        del halos_last_snap
        del subs_last_snap
        del gals_last_snap
        #gc.collect() # garbage collection -- safe but very slow.

        # Process the halos
        # The determination of timesteps could be done at initialisation (in update_parameters)
        t_step=((parameters.snap_table['time_in_years'][i_snap]- \
            parameters.snap_table['time_in_years'][i_snap-1]) * u.yr / parameters.units_time_internal).value
        n_dt_halo=int(t_step*1.000001/parameters.timestep_halo_internal)+1
        parameters.n_dt_halo=n_dt_halo
        parameters.dt_halo=t_step/n_dt_halo
        if VERBOSITY >= 2: print('t_step, n_dt_halo, dt_halo =',t_step, n_dt_halo,parameters.dt_halo)
        for i_dt in range(n_dt_halo):
            F_process_halos(halos_this_snap,subs_this_snap,gals_this_snap,graph,parameters)
            
        # Once all halos have been done, output results
        # This could instead be done on a halo-by-halo basis in F_process_halos
        halo_output.append(halos_this_snap,parameters)
        if subs_this_snap != None: sub_output.append(subs_this_snap,parameters)
        if isinstance(gals_this_snap, np.ndarray):
            gals_this_snap 
            gal_output.append(gals_this_snap,parameters)
            n_gal+=len(gals_this_snap)
            
        # Rename this_snap data structures to last_snap
        halos_last_snap=halos_this_snap
        subs_last_snap=subs_this_snap
        gals_last_snap=gals_this_snap

        # Delete old references (so that create new objects on next snapshot)
        del halos_this_snap
        del subs_this_snap
        del gals_this_snap

    # Tidy up
    del halos_last_snap
    del subs_last_snap
    del gals_last_snap


###  Tidy up and exit

In [ ]:
# Flush buffers, close files and exit
graph_file.close()
halo_output.close()
sub_output.close()
gal_output.close()

# Reopen galaxy output file to add graph start locations
# Don't simply do this before originally closing the output file, as the close flushes output buffers
gal_output=h5py.File(parameters.galaxy_file,'r+')
# Sanity check that have correct number of galaxies
assert n_gal==len(gal_output['Galaxies'])
# Add graph start locations as new dataset
dset=gal_output.create_dataset('Graph_start_locations',data=n_gal_graph_start,compression='gzip')
# And close
gal_output.close()